<a href="https://colab.research.google.com/github/Malanidhruv/quant-projects/blob/main/Stock_Fno_improved_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pya3 import Aliceblue
from nsetools import Nse
from datetime import datetime, timedelta
import holidays



In [10]:
def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key


In [11]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from pya3 import Aliceblue
from datetime import datetime
from nsetools import Nse

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Resample data to get monthly percentage change
    monthly_pct_change = df['pct'].resample('M').sum()
    df = monthly_pct_change.to_frame(name='pct')

    # Extract year and month for analysis
    df['year'] = df.index.year
    df['month'] = df.index.month

    # Calculate average monthly returns and standard deviation
    monthly_avg = df.groupby('month')['pct'].mean()
    monthly_std = df.groupby('month')['pct'].std()

    # Combine into a single DataFrame
    monthly_stats = pd.DataFrame({
        'Average Return': monthly_avg,
        'Standard Deviation': monthly_std
    }).sort_values(by='Average Return', ascending=False)

    return monthly_stats

def print_signals(top_stocks):
    print("\nStocks with July in Top 3 Months of Highest Average Return:")
    for stock_name, top_months, month_names in top_stocks:
        print(f"\n{stock_name}:")
        for month, name in zip(top_months.index, month_names):
            print(f"{name}: {top_months.loc[month, 'Average Return']:.2%}")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

def main():
    # Initialize Aliceblue object and get credentials
    alice = initialize_alice()

    # List of tokens to analyze
    tokens = [
        11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808,
        17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25,
        14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229,
        3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031,
        157, 1363, 16669, 3499
    ]

    top_stocks = []

    for token in tokens:
        monthly_stats = analyze_stock(alice, token)

        # Get top three months
        top_months = monthly_stats.head(3)
        month_names = top_months.index.map(lambda x: datetime(1900, x, 1).strftime('%B'))

        if 7 in top_months.index:  # Check if July (7) is one of the top 3 months
            stock_name = get_stock_name(alice, token)
            top_stocks.append((stock_name, top_months, month_names))

    # Print signals
    print_signals(top_stocks)

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: p45ZLNepIFnmQBWFVTqDU6qhmGsOppf0ZLOuOYyoafbyuMTDOd6pcK5kCn0KL5HclXfC7eG5CapFmbgt4F35PUZ9zk5ZC0F4e1GRlnmAT3O5v82VZJeD9PluY7Bfyqsb
NOTE: Today's contract master file will be updated after 08:00 AM. Before 08:00 AM previous day contract file be downloaded.

Stocks with July in Top 3 Months of Highest Average Return:

ULTRACEMCO-EQ:
July: 4.55%
October: 2.85%
January: 2.58%

HDFCLIFE-EQ:
April: 5.29%
November: 5.13%
July: 4.12%

RELIANCE-EQ:
April: 5.30%
July: 4.37%
June: 4.31%

HINDUNILVR-EQ:
May: 3.33%
June: 2.93%
July: 2.88%

BRITANNIA-EQ:
May: 4.53%
July: 4.20%
April: 3.81%

SBILIFE-EQ:
July: 7.28%
April: 5.27%
May: 3.40%

TCS-EQ:
July: 2.76%
April: 2.61%
December: 2.58%

TECHM-EQ:
August: 7.28%
July: 4.09%
November: 3.63%

ASIANPAINT-EQ:
July: 8.09%
December: 4.31%
May: 3.54%

ICICIBANK-EQ:
October: 8.49%
July: 4.78%
April: 4.60%

INDUSINDBK-EQ:
July: 6.19%
November: 5.23%
April: 4.27%

INFY-EQ:
July: 6.11%
January: 3.59%
November: 2.57%

In [12]:
import numpy as np
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Resample data to get monthly percentage change
    monthly_pct_change = df['pct'].resample('M').sum()
    df = monthly_pct_change.to_frame(name='pct')

    # Extract year and month for analysis
    df['year'] = df.index.year
    df['month'] = df.index.month

    # Calculate average monthly returns and standard deviation
    monthly_avg = df.groupby('month')['pct'].mean()
    monthly_std = df.groupby('month')['pct'].std()

    # Combine into a single DataFrame
    monthly_stats = pd.DataFrame({
        'Average Return': monthly_avg,
        'Standard Deviation': monthly_std
    }).sort_values(by='Average Return', ascending=False)

    return monthly_stats

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

def main():
    # Initialize Aliceblue object and get credentials
    alice = initialize_alice()

    # List of tokens to analyze
    tokens = [
        11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808,
        17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25,
        14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229,
        3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031,
        157, 1363, 16669, 3499
    ]

    top_tokens = []

    for token in tokens:
        monthly_stats = analyze_stock(alice, token)

        # Get top three months
        top_months = monthly_stats.head(3)

        if 7 in top_months.index:  # Check if July (7) is one of the top 3 months
            stock_name = get_stock_name(alice, token)
            top_tokens.append((token, stock_name))

    # Print tokens and stock names
    print("\nTokens and Stocks with July in Top 3 Months of Highest Average Return:")
    for token, stock_name in top_tokens:
        print(f"Token: {token}, Stock Name: {stock_name}")

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: p45ZLNepIFnmQBWFVTqDU6qhmGsOppf0ZLOuOYyoafbyuMTDOd6pcK5kCn0KL5HclXfC7eG5CapFmbgt4F35PUZ9zk5ZC0F4e1GRlnmAT3O5v82VZJeD9PluY7Bfyqsb

Tokens and Stocks with July in Top 3 Months of Highest Average Return:
Token: 11532, Stock Name: ULTRACEMCO-EQ
Token: 467, Stock Name: HDFCLIFE-EQ
Token: 2885, Stock Name: RELIANCE-EQ
Token: 1394, Stock Name: HINDUNILVR-EQ
Token: 547, Stock Name: BRITANNIA-EQ
Token: 21808, Stock Name: SBILIFE-EQ
Token: 11536, Stock Name: TCS-EQ
Token: 13538, Stock Name: TECHM-EQ
Token: 236, Stock Name: ASIANPAINT-EQ
Token: 4963, Stock Name: ICICIBANK-EQ
Token: 5258, Stock Name: INDUSINDBK-EQ
Token: 1594, Stock Name: INFY-EQ
Token: 10940, Stock Name: DIVISLAB-EQ
Token: 3351, Stock Name: SUNPHARMA-EQ
Token: 3787, Stock Name: WIPRO-EQ
Token: 3432, Stock Name: TATACONSUM-EQ
Token: 317, Stock Name: BAJFINANCE-EQ
Token: 7229, Stock Name: HCLTECH-EQ
Token: 3045, Stock Name: SBIN-EQ
Token: 16675, Stock Name: BAJAJFINSV-EQ
Token: 11723,

In [13]:
import numpy as np
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Resample data to get monthly percentage change
    monthly_pct_change = df['pct'].resample('M').sum()
    df = monthly_pct_change.to_frame(name='pct')

    # Extract year and month for analysis
    df['year'] = df.index.year
    df['month'] = df.index.month

    # Calculate average monthly returns and standard deviation
    monthly_avg = df.groupby('month')['pct'].mean()
    monthly_std = df.groupby('month')['pct'].std()

    # Combine into a single DataFrame
    monthly_stats = pd.DataFrame({
        'Average Return': monthly_avg,
        'Standard Deviation': monthly_std
    }).sort_values(by='Average Return', ascending=False)

    return monthly_stats

def analyze_july_categories(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Filter for July data
    july_data = df[df.index.month == 7]

    # Create date categories
    def categorize_date(date):
        if 1 <= date.day <= 10:
            return 'Category 1 (1-10)'
        elif 11 <= date.day <= 20:
            return 'Category 2 (11-20)'
        else:
            return 'Category 3 (21-31)'

    july_data['category'] = july_data.index.to_series().apply(categorize_date)

    # Calculate percentage change for each category
    category_pct_change = july_data.groupby('category')['pct'].sum()

    return category_pct_change

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

def main():
    # Initialize Aliceblue object and get credentials
    alice = initialize_alice()

    # List of tokens to analyze
    tokens = [
        11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808,
        17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25,
        14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229,
        3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031,
        157, 1363, 16669, 3499
    ]

    top_tokens = []

    for token in tokens:
        monthly_stats = analyze_stock(alice, token)

        # Get top three months
        top_months = monthly_stats.head(3)

        if 7 in top_months.index:  # Check if July (7) is one of the top 3 months
            stock_name = get_stock_name(alice, token)
            top_tokens.append((token, stock_name))

    # Print tokens and stock names
    print("\nTokens and Stocks with July in Top 3 Months of Highest Average Return:")
    for token, stock_name in top_tokens:
        print(f"Token: {token}, Stock Name: {stock_name}")

    # Analyze July categories for top stocks
    print("\nJuly Category Analysis for Top Stocks:")
    for token, stock_name in top_tokens:
        category_pct_change = analyze_july_categories(alice, token)
        print(f"\n{stock_name} (Token: {token}):")
        for category, pct_change in category_pct_change.items():
            print(f"{category}: {pct_change:.2%}")

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: p45ZLNepIFnmQBWFVTqDU6qhmGsOppf0ZLOuOYyoafbyuMTDOd6pcK5kCn0KL5HclXfC7eG5CapFmbgt4F35PUZ9zk5ZC0F4e1GRlnmAT3O5v82VZJeD9PluY7Bfyqsb

Tokens and Stocks with July in Top 3 Months of Highest Average Return:
Token: 11532, Stock Name: ULTRACEMCO-EQ
Token: 467, Stock Name: HDFCLIFE-EQ
Token: 2885, Stock Name: RELIANCE-EQ
Token: 1394, Stock Name: HINDUNILVR-EQ
Token: 547, Stock Name: BRITANNIA-EQ
Token: 21808, Stock Name: SBILIFE-EQ
Token: 11536, Stock Name: TCS-EQ
Token: 13538, Stock Name: TECHM-EQ
Token: 236, Stock Name: ASIANPAINT-EQ
Token: 4963, Stock Name: ICICIBANK-EQ
Token: 5258, Stock Name: INDUSINDBK-EQ
Token: 1594, Stock Name: INFY-EQ
Token: 10940, Stock Name: DIVISLAB-EQ
Token: 3351, Stock Name: SUNPHARMA-EQ
Token: 3787, Stock Name: WIPRO-EQ
Token: 3432, Stock Name: TATACONSUM-EQ
Token: 317, Stock Name: BAJFINANCE-EQ
Token: 7229, Stock Name: HCLTECH-EQ
Token: 3045, Stock Name: SBIN-EQ
Token: 16675, Stock Name: BAJAJFINSV-EQ
Token: 11723,

<ipython-input-13-9097fc5358ea>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



ULTRACEMCO-EQ (Token: 11532):
Category 1 (1-10): 12.49%
Category 2 (11-20): 24.34%
Category 3 (21-31): 13.18%


<ipython-input-13-9097fc5358ea>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



HDFCLIFE-EQ (Token: 467):
Category 1 (1-10): 12.77%
Category 2 (11-20): 6.36%
Category 3 (21-31): 9.72%


...

<ipython-input-13-9097fc5358ea>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



JSWSTEEL-EQ (Token: 11723):
Category 1 (1-10): -5.83%
Category 2 (11-20): 19.66%
Category 3 (21-31): 24.74%


<ipython-input-13-9097fc5358ea>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



CIPLA-EQ (Token: 694):
Category 1 (1-10): 9.61%
Category 2 (11-20): 21.82%
Category 3 (21-31): 20.74%

HINDALCO-EQ (Token: 1363):
Category 1 (1-10): 16.28%
Category 2 (11-20): 14.81%
Category 3 (21-31): 41.46%


<ipython-input-13-9097fc5358ea>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)


In [14]:
import numpy as np
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Resample data to get monthly percentage change
    monthly_pct_change = df['pct'].resample('M').sum()
    df = monthly_pct_change.to_frame(name='pct')

    # Extract year and month for analysis
    df['year'] = df.index.year
    df['month'] = df.index.month

    # Calculate average monthly returns and standard deviation
    monthly_avg = df.groupby('month')['pct'].mean()
    monthly_std = df.groupby('month')['pct'].std()

    # Combine into a single DataFrame
    monthly_stats = pd.DataFrame({
        'Average Return': monthly_avg,
        'Standard Deviation': monthly_std
    }).sort_values(by='Average Return', ascending=False)

    return monthly_stats

def analyze_july_categories(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Filter for July data
    july_data = df[df.index.month == 7]

    # Create date categories
    def categorize_date(date):
        if 1 <= date.day <= 10:
            return 'Category 1 (1-10)'
        elif 11 <= date.day <= 20:
            return 'Category 2 (11-20)'
        else:
            return 'Category 3 (21-31)'

    july_data['category'] = july_data.index.to_series().apply(categorize_date)

    # Calculate average percentage change for each category over the years
    category_avg_pct_change = july_data.groupby(['category', july_data.index.year])['pct'].sum().groupby('category').mean()

    return category_avg_pct_change

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

def main():
    # Initialize Aliceblue object and get credentials
    alice = initialize_alice()

    # List of tokens to analyze
    tokens = [
        11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808,
        17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25,
        14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229,
        3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031,
        157, 1363, 16669, 3499
    ]

    top_tokens = []

    for token in tokens:
        monthly_stats = analyze_stock(alice, token)

        # Get top three months
        top_months = monthly_stats.head(3)

        if 7 in top_months.index:  # Check if July (7) is one of the top 3 months
            stock_name = get_stock_name(alice, token)
            top_tokens.append((token, stock_name))

    # Print tokens and stock names
    print("\nTokens and Stocks with July in Top 3 Months of Highest Average Return:")
    for token, stock_name in top_tokens:
        print(f"Token: {token}, Stock Name: {stock_name}")

    # Analyze July categories for top stocks
    print("\nJuly Category Analysis for Top Stocks:")
    for token, stock_name in top_tokens:
        category_avg_pct_change = analyze_july_categories(alice, token)
        print(f"\n{stock_name} (Token: {token}):")
        for category, avg_pct_change in category_avg_pct_change.items():
            print(f"{category}: {avg_pct_change:.2%}")

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: p45ZLNepIFnmQBWFVTqDU6qhmGsOppf0ZLOuOYyoafbyuMTDOd6pcK5kCn0KL5HclXfC7eG5CapFmbgt4F35PUZ9zk5ZC0F4e1GRlnmAT3O5v82VZJeD9PluY7Bfyqsb

Tokens and Stocks with July in Top 3 Months of Highest Average Return:
Token: 11532, Stock Name: ULTRACEMCO-EQ
Token: 467, Stock Name: HDFCLIFE-EQ
Token: 2885, Stock Name: RELIANCE-EQ
Token: 1394, Stock Name: HINDUNILVR-EQ
Token: 547, Stock Name: BRITANNIA-EQ
Token: 21808, Stock Name: SBILIFE-EQ
Token: 11536, Stock Name: TCS-EQ
Token: 13538, Stock Name: TECHM-EQ
Token: 236, Stock Name: ASIANPAINT-EQ
Token: 4963, Stock Name: ICICIBANK-EQ
Token: 5258, Stock Name: INDUSINDBK-EQ
Token: 1594, Stock Name: INFY-EQ
Token: 10940, Stock Name: DIVISLAB-EQ
Token: 3351, Stock Name: SUNPHARMA-EQ
Token: 3787, Stock Name: WIPRO-EQ
Token: 3432, Stock Name: TATACONSUM-EQ
Token: 317, Stock Name: BAJFINANCE-EQ
Token: 7229, Stock Name: HCLTECH-EQ
Token: 3045, Stock Name: SBIN-EQ
Token: 16675, Stock Name: BAJAJFINSV-EQ
Token: 11723,

<ipython-input-14-51d15ba71ef7>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



ULTRACEMCO-EQ (Token: 11532):
Category 1 (1-10): 1.14%
Category 2 (11-20): 2.43%
Category 3 (21-31): 1.32%


<ipython-input-14-51d15ba71ef7>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



HDFCLIFE-EQ (Token: 467):
Category 1 (1-10): 1.82%
Category 2 (11-20): 1.06%
Category 3 (21-31): 1.62%


...

<ipython-input-14-51d15ba71ef7>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



JSWSTEEL-EQ (Token: 11723):
Category 1 (1-10): -0.53%
Category 2 (11-20): 1.97%
Category 3 (21-31): 2.47%


<ipython-input-14-51d15ba71ef7>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



CIPLA-EQ (Token: 694):
Category 1 (1-10): 0.87%
Category 2 (11-20): 2.18%
Category 3 (21-31): 2.07%

HINDALCO-EQ (Token: 1363):
Category 1 (1-10): 1.48%
Category 2 (11-20): 1.48%
Category 3 (21-31): 4.15%


<ipython-input-14-51d15ba71ef7>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)


In [15]:
import numpy as np
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Resample data to get monthly percentage change
    monthly_pct_change = df['pct'].resample('M').sum()
    df = monthly_pct_change.to_frame(name='pct')

    # Extract year and month for analysis
    df['year'] = df.index.year
    df['month'] = df.index.month

    # Calculate average monthly returns and standard deviation
    monthly_avg = df.groupby('month')['pct'].mean()
    monthly_std = df.groupby('month')['pct'].std()

    # Combine into a single DataFrame
    monthly_stats = pd.DataFrame({
        'Average Return': monthly_avg,
        'Standard Deviation': monthly_std
    }).sort_values(by='Average Return', ascending=False)

    return monthly_stats

def analyze_july_categories(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Filter for July data
    july_data = df[df.index.month == 7]

    # Create date categories
    def categorize_date(date):
        if 1 <= date.day <= 10:
            return 'Category 1 (1-10)'
        elif 11 <= date.day <= 20:
            return 'Category 2 (11-20)'
        else:
            return 'Category 3 (21-31)'

    july_data['category'] = july_data.index.to_series().apply(categorize_date)

    # Calculate average percentage change for each category over the years
    category_avg_pct_change = july_data.groupby(['category', july_data.index.year])['pct'].sum().groupby('category').mean()

    return category_avg_pct_change

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

def main():
    # Initialize Aliceblue object and get credentials
    alice = initialize_alice()

    # List of tokens to analyze
    tokens = [
        11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808,
        17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25,
        14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229,
        3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031,
        157, 1363, 16669, 3499
    ]

    top_tokens = []

    for token in tokens:
        monthly_stats = analyze_stock(alice, token)

        # Get top three months
        top_months = monthly_stats.head(3)

        if 7 in top_months.index:  # Check if July (7) is one of the top 3 months
            stock_name = get_stock_name(alice, token)
            top_tokens.append((token, stock_name))

    # Print tokens and stock names
    print("\nTokens and Stocks with July in Top 3 Months of Highest Average Return:")
    for token, stock_name in top_tokens:
        print(f"Token: {token}, Stock Name: {stock_name}")

    # Analyze July categories for top stocks
    print("\nJuly Category Analysis for Top Stocks:")
    for token, stock_name in top_tokens:
        category_avg_pct_change = analyze_july_categories(alice, token)
        print(f"\n{stock_name} (Token: {token}):")
        for category, avg_pct_change in category_avg_pct_change.items():
            print(f"{category} (Dates {category.split('(')[1].split(')')[0]}): {avg_pct_change:.2%}")

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: p45ZLNepIFnmQBWFVTqDU6qhmGsOppf0ZLOuOYyoafbyuMTDOd6pcK5kCn0KL5HclXfC7eG5CapFmbgt4F35PUZ9zk5ZC0F4e1GRlnmAT3O5v82VZJeD9PluY7Bfyqsb

Tokens and Stocks with July in Top 3 Months of Highest Average Return:
Token: 11532, Stock Name: ULTRACEMCO-EQ
Token: 467, Stock Name: HDFCLIFE-EQ
Token: 2885, Stock Name: RELIANCE-EQ
Token: 1394, Stock Name: HINDUNILVR-EQ
Token: 547, Stock Name: BRITANNIA-EQ
Token: 21808, Stock Name: SBILIFE-EQ
Token: 11536, Stock Name: TCS-EQ
Token: 13538, Stock Name: TECHM-EQ
Token: 236, Stock Name: ASIANPAINT-EQ
Token: 4963, Stock Name: ICICIBANK-EQ
Token: 5258, Stock Name: INDUSINDBK-EQ
Token: 1594, Stock Name: INFY-EQ
Token: 10940, Stock Name: DIVISLAB-EQ
Token: 3351, Stock Name: SUNPHARMA-EQ
Token: 3787, Stock Name: WIPRO-EQ
Token: 3432, Stock Name: TATACONSUM-EQ
Token: 317, Stock Name: BAJFINANCE-EQ
Token: 7229, Stock Name: HCLTECH-EQ
Token: 3045, Stock Name: SBIN-EQ
Token: 16675, Stock Name: BAJAJFINSV-EQ
Token: 11723,

<ipython-input-15-6a3937d09b73>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



ULTRACEMCO-EQ (Token: 11532):
Category 1 (1-10) (Dates 1-10): 1.14%
Category 2 (11-20) (Dates 11-20): 2.43%
Category 3 (21-31) (Dates 21-31): 1.32%


<ipython-input-15-6a3937d09b73>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



HDFCLIFE-EQ (Token: 467):
Category 1 (1-10) (Dates 1-10): 1.82%
Category 2 (11-20) (Dates 11-20): 1.06%
Category 3 (21-31) (Dates 21-31): 1.62%


...

<ipython-input-15-6a3937d09b73>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



JSWSTEEL-EQ (Token: 11723):
Category 1 (1-10) (Dates 1-10): -0.53%
Category 2 (11-20) (Dates 11-20): 1.97%
Category 3 (21-31) (Dates 21-31): 2.47%


<ipython-input-15-6a3937d09b73>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)



CIPLA-EQ (Token: 694):
Category 1 (1-10) (Dates 1-10): 0.87%
Category 2 (11-20) (Dates 11-20): 2.18%
Category 3 (21-31) (Dates 21-31): 2.07%

HINDALCO-EQ (Token: 1363):
Category 1 (1-10) (Dates 1-10): 1.48%
Category 2 (11-20) (Dates 11-20): 1.48%
Category 3 (21-31) (Dates 21-31): 4.15%


<ipython-input-15-6a3937d09b73>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)


In [17]:
import numpy as np
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime
from fpdf import FPDF

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Resample data to get monthly percentage change
    monthly_pct_change = df['pct'].resample('M').sum()
    df = monthly_pct_change.to_frame(name='pct')

    # Extract year and month for analysis
    df['year'] = df.index.year
    df['month'] = df.index.month

    # Calculate average monthly returns and standard deviation
    monthly_avg = df.groupby('month')['pct'].mean()
    monthly_std = df.groupby('month')['pct'].std()

    # Combine into a single DataFrame
    monthly_stats = pd.DataFrame({
        'Average Return': monthly_avg,
        'Standard Deviation': monthly_std
    }).sort_values(by='Average Return', ascending=False)

    return monthly_stats

def analyze_july_categories(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 1)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Filter for July data
    july_data = df[df.index.month == 7]

    # Create date categories
    def categorize_date(date):
        if 1 <= date.day <= 10:
            return 'Category 1 (1-10)'
        elif 11 <= date.day <= 20:
            return 'Category 2 (11-20)'
        else:
            return 'Category 3 (21-31)'

    july_data['category'] = july_data.index.to_series().apply(categorize_date)

    # Calculate average percentage change for each category over the years
    category_avg_pct_change = july_data.groupby(['category', july_data.index.year])['pct'].sum().groupby('category').mean()

    return category_avg_pct_change

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

def save_to_pdf(report, filename='July_Stock_Analysis.pdf'):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    for line in report:
        pdf.cell(200, 10, txt=line, ln=True, align='L')

    pdf.output(filename)

def main():
    # Initialize Aliceblue object and get credentials
    alice = initialize_alice()

    # List of tokens to analyze
    tokens = [
        11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808,
        17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25,
        14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229,
        3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031,
        157, 1363, 16669, 3499
    ]

    top_tokens = []

    for token in tokens:
        monthly_stats = analyze_stock(alice, token)

        # Get top three months
        top_months = monthly_stats.head(3)

        if 7 in top_months.index:  # Check if July (7) is one of the top 3 months
            stock_name = get_stock_name(alice, token)
            top_tokens.append((token, stock_name))

    # Collect report data
    report = []

    report.append("Tokens and Stocks with July in Top 3 Months of Highest Average Return:")
    for token, stock_name in top_tokens:
        report.append(f"Token: {token}, Stock Name: {stock_name}")

    report.append("\nJuly Category Analysis for Top Stocks:")
    for token, stock_name in top_tokens:
        category_avg_pct_change = analyze_july_categories(alice, token)
        report.append(f"\n{stock_name} (Token: {token}):")
        for category, avg_pct_change in category_avg_pct_change.items():
            report.append(f"{category} (Dates {category.split('(')[1].split(')')[0]}): {avg_pct_change:.2%}")

    # Save the report to PDF
    save_to_pdf(report)

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: p45ZLNepIFnmQBWFVTqDU6qhmGsOppf0ZLOuOYyoafbyuMTDOd6pcK5kCn0KL5HclXfC7eG5CapFmbgt4F35PUZ9zk5ZC0F4e1GRlnmAT3O5v82VZJeD9PluY7Bfyqsb


<ipython-input-17-e71320cf3267>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)
<ipython-input-17-e71320cf3267>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_data['category'] = july_data.index.to_series().apply(categorize_date)
<ipython-input-17-e71320cf3267>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea